In [ ]:
%pip install spacy

In [ ]:
!python -m spacy download pt_core_news_lg


In [ ]:
%pip install nltk
%pip install spacy-wordnet


## Aprendendo sobre synset e lemmas 

In [33]:

import spacy
import nltk
from nltk.corpus import wordnet

nltk.download('wordnet')
nltk.download('rslp')

# Carregue o modelo SpaCy
nlp = spacy.load('pt_core_news_lg')

# Adicione o WordnetAnnotator ao pipeline de processamento
nlp.add_pipe('spacy_wordnet', after='lemmatizer')


doc = nlp("O jogador de futebol chutou a bola para o gol.")
stemmer = nltk.stem.RSLPStemmer()

for token in doc:
    print(token.text, "|", token.lemma_,"|", stemmer.stem(token.text))




[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\drc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\drc\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


O | o | o
jogador | jogador | jog
de | de | de
futebol | futebol | futebol
chutou | chutar | chut
a | o | a
bola | bola | bol
para | para | par
o | o | o
gol | gol | gol
. | . | .


In [40]:
!python -m nltk.downloader wordnet
!python -m nltk.downloader omw

c:\Users\drc\AppData\Local\anaconda3\envs\py38\lib\runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\drc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
c:\Users\drc\AppData\Local\anaconda3\envs\py38\lib\runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package omw to
[nltk_data]     C:\Users\drc\AppData\Roaming\nltk_data...
[nltk_data]   Package omw is already up-to-date!


In [ ]:
%pip install spacy-wordnet


## Aprendendo sobre o space_wordnet

spaCy Wordnet is a simple custom component for using WordNet, MultiWordnet and WordNet domains with spaCy.

Get all synsets for a processed token. For example, getting all the synsets (word senses) of the word bank.
Get and filter synsets by domain. For example, getting synonyms of the verb withdraw in the financial domain.

In [45]:
import spacy
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 


nlp = spacy.load('pt_core_news_lg')
nlp.add_pipe("spacy_wordnet", after='lemmatizer')


text = "Eu vou cozinhar batatas fritas."
economy_domains = ['food', 'cooking']
enriched_sentence = []
sentence = nlp(text)

stemmer = nltk.stem.RSLPStemmer()

for token in doc:
    print(token.text, "|", token.lemma_,"|", stemmer.stem(token.text))


for token in sentence:
    # We get those synsets within the desired domains
    synsets = token._.wordnet.wordnet_synsets_for_domain(economy_domains)
    if not synsets:
        enriched_sentence.append(token.text)
    else:
        lemmas_for_synset = [lemma for s in synsets for lemma in s.lemma_names('por')]
        # If we found a synset in the economy domains
        # we get the variants and add them to the enriched sentence
        enriched_sentence.append('({})'.format('|'.join(set(lemmas_for_synset))))

# Let's see our enriched sentence
print(' '.join(enriched_sentence))

Eu vou (cozinhar|preparar) (marmêndoa|batata|batatinha) fritas .


## Transformando os pdfs em texto em um arquivo

In [47]:

import os
from PyPDF2 import PdfReader

folder_path = "./pdf"
output_folder = "./output_text"

# Certifique-se de que o diretório de saída exista, se não existir, crie-o
os.makedirs(output_folder, exist_ok=True)

# Para cada arquivo em folder_path
for filename in os.listdir(folder_path):
    # Verificar se termina com .pdf
    if filename.lower().endswith(".pdf"):
        # Pegar o path do arquivo
        pdf_path = os.path.join(folder_path, filename)

        # Criar um contexto para garantir que o arquivo será fechado
        with open(pdf_path, 'rb') as f:
            # Criar um leitor de PDF
            pdf = PdfReader(f)

            # Inicializar o texto do PDF
            pdf_text = ""

            # Iterar sobre as páginas e concatenar o texto
            for page in pdf.pages:
                pdf_text += page.extract_text()

            # Criar o nome do arquivo de saída
            output_file_name = f"{filename}_completo.txt"
            output_file_path = os.path.join(output_folder, output_file_name)

            # Escrever o texto completo em um arquivo de saída
            with open(output_file_path, 'w', encoding='utf-8') as output_file:
                output_file.write(pdf_text)



## Tentando fazer sinonimos para cada token dos pdf


In [80]:
# Importando bibliotecas
import os  # Para interagir com o sistema operacional
import spacy  # Para processamento de linguagem natural
import nltk  # Toolkit de processamento de linguagem natural
from nltk.corpus import wordnet  # Corpus do WordNet da NLTK, blibioteca que contem palavras 
from spacy_wordnet.wordnet_annotator import WordnetAnnotator  # Anotador WordNet para o SpaCy

#Adiciona anotações relacionadas ao WordNet ao pipeline do SpaCy. No código, é usado para obter informações do WordNet para os tokens do SpaCy.

nltk.download('wordnet')

# Caminho para os pdf em formato de texto
folder_path = './output_text/'
# Caminho para o arquivo de saída
output_file_path = 'sinonimos_output.txt'

# Carregar o modelo SpaCy
nlp = spacy.load('pt_core_news_lg')
# Adicionar o Anotador Wordnet ao pipeline do SpaCy
nlp.add_pipe("spacy_wordnet", after='lemmatizer')

# Inicializar o stemmer da NLTK
stemmer = nltk.stem.RSLPStemmer()


# Lista para armazenar os resultados
resultados = []

# Função para obter sinônimos de uma palavra
def get_synonyms(word, pos=wordnet.NOUN):
    synonyms = set()
    for syn in wordnet.synsets(word, pos=pos):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

# Iterar sobre os arquivos no diretório de saída
for filename in os.listdir(folder_path):
     # Ler o conteúdo do arquivo PDF
    doc = nlp(open(os.path.join(folder_path, filename), encoding="utf-8").read())

    # Lista para armazenar os resultados para cada documento
    document_result = []

     # Iterar sobre os tokens no documento
    for token in doc:

        # Obter todos os synsets para o token
        synsets = token._.wordnet.synsets()
        
        # Lista para armazenar os lemmas e sinônimos para cada token
        token_result = []

         # Se não houver synsets, adicione o texto original
        if not synsets:
            token_result.append(token.text)
        else:
             # Obter lemmas para cada synset
            lemmas_for_synset = [lemma for s in synsets for lemma in s.lemma_names('por')]
            # Obter sinônimos para cada lemma
            synonyms_for_synset = set()
            for synset in synsets:
                for lemma in synset.lemma_names('por'):
                    synonyms_for_synset.update(get_synonyms(lemma, pos=wordnet.NOUN))
            # Adicionar lemmas e sinônimos ao resultado
            token_result.append('({})'.format('|'.join(set(lemmas_for_synset + list(synonyms_for_synset)))))
        
        # Adicionar informações a resultados para cada token
        document_result.append(f'{token.text} | {token.lemma_} | {stemmer.stem(token.text)} | {" ".join(token_result)}')
        

    # Adicionar resultados para o documento atual à lista geral
    resultados.extend(document_result)

# Escrever os resultados em um novo arquivo
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    output_file.write('\n'.join(resultados))


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\drc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [101]:
import os  # Para interagir com o sistema operacional
import spacy  # Para processamento de linguagem natural
import nltk  # Toolkit de processamento de linguagem natural

#classes do spacy que relacioandos ao wordnet que é uma coleção de palavras 
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 
from spacy.language import Language

#o wordnet é uma coleção de sinônimos e antônimos de palavras, enquanto o corpus OWM é uma versão multilíngue do WordNet 
nltk.download('wordnet')
nltk.download('omw')

#carrega o modelo pre-treinado do spacy 
nlp = spacy.load('pt_core_news_lg')

#A classe Language é uma classe de alto nível do SpaCy que representa o idioma e seu modelo. 
#Você pode adicionar processadores personalizados ao pipeline de processamento de linguagem natural do SpaCy usando a classe
@Language.component("wordnet_annotator")
def add_wordnet_annotator(doc):
    wordnet = WordnetAnnotator(nlp, "wordnet")
    doc = wordnet(doc)
    return doc

nlp.add_pipe("wordnet_annotator", name="wordnet", last=True)


def has_synonyms(token):
    return len(token._.synonyms) > 0

def synonyms(token, synonyms):
    token._.synonyms = synonyms

spacy.tokens.Token.set_extension("has_synonyms", getter=has_synonyms, force=True)
spacy.tokens.Token.set_extension("set_synonyms", method=synonyms, force=True)
spacy.tokens.Token.set_extension("synonyms", default=[], force=True)

folder_path = "./output_text/"

for filename in os.listdir(folder_path):
    with open(os.path.join(folder_path, filename), encoding="utf-8") as file:
        text = file.read()
        doc = nlp(text)

        infos_subs = []
        substantivos = set()

        for token in doc:
            if token.pos_ == "NOUN" and token.text not in substantivos and token.text and token.text.isalpha():
                
                if token._.wordnet is not None:
                    print(token.text)
                    synsets = token._.wordnet.synsets()
                    print("sysnets: ", synsets)
                    lemmas = [lemma for s in synsets for lemma in s.lemmas('por')]
                    print("lemmas: ", lemmas_for_synset)
                    synonyms = [lemma.name() for lemma in lemmas]
                    print("synonyms: ", synonyms)
                    token._.set_synonyms(synonyms)
                
                sinonimos =  ", ".join([synonym for synonym in token._.synonyms]) if token._.has_synonyms else ""
                infos_subs.append(f"{token.text}  | {sinonimos} ")
                substantivos.add(token.text)
                

output_file_path = 'sinonimos_output-2.txt'
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    output_file.write('\n'.join(infos_subs))

print("Arquivos processados: ", {filename})



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\drc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to
[nltk_data]     C:\Users\drc\AppData\Roaming\nltk_data...
[nltk_data]   Package omw is already up-to-date!


TÍTULO
sysnets:  [Synset('appellation.n.01'), Synset('title.n.06'), Synset('title.n.02'), Synset('title.n.03'), Synset('caption.n.03')]
lemmas:  ['consulta', 'consulta']
synonyms:  ['antenome', 'denominação', 'nome', 'nome_de_batismo', 'nome_próprio', 'prenome', 'primeiro_nome', 'título', 'título', 'título', 'título', 'legenda', 'lenda', 'subtítulo', 'título']
N
sysnets:  [Synset('n.n.05')]
lemmas:  ['consulta', 'consulta']
synonyms:  ['n']
ÍNDICE
sysnets:  [Synset('contents.n.01'), Synset('index.n.02')]
lemmas:  ['consulta', 'consulta']
synonyms:  ['conteúdo', 'índice', 'Tabela_de_conteúdo', 'índice']
REVISÕES
sysnets:  []
lemmas:  ['consulta', 'consulta']
synonyms:  []
DESCRIÇÃO
sysnets:  [Synset('description.n.03'), Synset('statement.n.01'), Synset('statement.n.01'), Synset('description.n.01'), Synset('description.n.02'), Synset('word_picture.n.01'), Synset('portrayal.n.01')]
lemmas:  ['consulta', 'consulta']
synonyms:  ['descrição', 'declaração', 'descrição', 'Descrição', 'declaraç